In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
import tensorflow as tf
keras = tf.keras
import cv2
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

sys.path.append("/home/files/vision_tree")
from utils.session_config import setup_gpus

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
setup_gpus(memory_fraction=0.2)

1 Physical GPUs, 1 Logical GPUs


In [4]:
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, ReLU, BatchNormalization, GlobalAveragePooling2D
from utils.losses import L2_Orthogonal
from utils.BEP import Back_etching_propagation

In [5]:
data_0 = np.load("/home/files/datasets/mnist/0.npy")
label_0 = np.zeros(len(data_0), np.int32)
data_5 = np.load("/home/files/datasets/mnist/5.npy")
label_5 = np.ones(len(data_5), np.int32)
data_7 = np.load("/home/files/datasets/mnist/7.npy")
label_7 = np.ones(len(data_7), np.int32)*2

train_x = tf.concat([data_0, data_5, data_7], axis=0)
train_x = tf.cast(train_x, tf.float32)
train_x = train_x[:,:,:,tf.newaxis]/255.
train_y = tf.concat([label_0, label_5, label_7], axis=0)

print(train_x.shape)
print(train_y.shape)

x = tf.data.Dataset.from_tensor_slices(train_x)
y = tf.data.Dataset.from_tensor_slices(train_y)
ds = tf.data.Dataset.zip((x, y))
ds = ds.shuffle(len(train_x), reshuffle_each_iteration=False).batch(32)
#ds = ds.shuffle(len(train_x), reshuffle_each_iteration=False).shuffle(1000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

(17609, 28, 28, 1)
(17609,)


In [6]:
trained_model = tf.keras.models.load_model("/home/files/vision_tree/train/ckpt/mnist_cnn/057.h5")
bep_algorithm = Back_etching_propagation(trained_model)
#etching start
bep_algorithm.categorical_etching(ds, num_class=3, save_path='057')

2 layer name :  conv2d_3
Conv2D
0 layer name :  re_lu_3
0 layer name :  max_pooling2d_2
2 layer name :  conv2d_4
Conv2D
0 layer name :  re_lu_4
0 layer name :  max_pooling2d_3
2 layer name :  conv2d_5
Conv2D
0 layer name :  re_lu_5
0 layer name :  global_average_pooling2d_1
2 layer name :  dense_1
Dense
17
12
7
2


In [10]:
import pandas as pd
import plotly
from utils.visualization import Feature, Vision_tree

L3,L2,L1,L0 = np.load("057/3.npy"),\
              np.load("057/2.npy"),\
              np.load("057/1.npy"),\
              np.load("057/0.npy")

In [11]:
vt = Vision_tree('mnist', [L1,L2,L3], 4)
vt.create_layers()
nodes, connections = vt.df_tree()

In [12]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Scatter3d(x=nodes['x'], y=nodes['y'], z=nodes['z'],
                                   mode='markers', marker=dict(
                                                            size=6,
                                                            color=nodes['color'],   # set color to an array/list of desired values
                                                            colorscale='Viridis',   # choose a colorscale
                                                            opacity=1.0))])

for connection in connections.values:
    x1,y1,z1,x2,y2,z2,color = connection
    fig.add_trace(go.Scatter3d(
            x=[x1,x2],
            y=[y1,y2],
            z=[z1,z2],
            mode='lines',
            line=dict(
        color=color,
        width=2
    )))

fig.show()